In [4]:
news_dict = {'FOMC': ['Manufacturing PMI', 'Non-Manufacturing PMI', 'CPI', 'Retail Sales', 'Trading Balance','Non-Farm Payroll'],
             'Non-Farm Payroll' : ['Manufacturing PMI', 'Non-Manufacturing PMI', 'CPI', 'Retail Sales', 'Trading Balance', 'FOMC Rates']
            }

In [5]:
keys = list(news_dict)

In [12]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import re

current_month = datetime.now().month
previous_month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'][current_month-2]

my_url = 'https://www.forexfactory.com/calendar?month=' + previous_month + '.2021'

PATH = 'C:\Program Files (x86)\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(PATH, options=options)
driver.get(my_url)
driver.get(driver.current_url)

html = driver.page_source
driver.quit()

page_soup = soup(html, "lxml")

In [32]:
initial_date = None

# This function checks if the date cell on the table is empty
# If it is not empty the global variable initial_date is assigned the value 0f the date
# If it is empty the date is assigned to the value of the initial_date
def fill_empty_date(current):
    global initial_date
    
    if current != None:
        initial_date = current
    else:
        current = initial_date
    return current

In [14]:
# This function converts a string into a datetime object.
def to_datetime(date):
    date = date + ' ' + str(datetime.now().year)
    return datetime.strptime(date , '%b %d %Y')

In [35]:
# This function removes all characters from a string except numbers and period and converts the string into a float.
def to_float(actual):
    actual = re.sub('[^0-9.]', '', actual)
    return float(actual)

In [19]:
expected_news = 'Non-Farm Payroll'
expected_currency = 'USD'

In [33]:
table = page_soup.find("table", {"class": "calendar__table"})

news_list = []

for row in table.select('tr.calendar__row.calendar_row.calendar__row--grey'):
    
    date = row.select_one('span.date span')
    date = fill_empty_date(date).text

    title = row.select_one('span.calendar__event-title').text
    
    if not any(n.lower() in title.lower() for n in news_dict.get(expected_news, [])):
        continue
        
    currency = row.select_one('td.calendar__cell.calendar__currency.currency').text.strip()
     
    if currency.lower() != expected_currency.lower():
        continue
    
    actual = row.select_one('td.calendar__cell.calendar__actual.actual').text
    
    news_list.append({'Title': title, 'Date': to_datetime(date), 'Actual': to_float(actual)})

In [34]:
news_list

[{'Title': 'Final Manufacturing PMI',
  'Date': datetime.datetime(2021, 4, 1, 0, 0),
  'Actual': 59.1},
 {'Title': 'ISM Manufacturing PMI',
  'Date': datetime.datetime(2021, 4, 1, 0, 0),
  'Actual': 64.7},
 {'Title': 'CPI m/m',
  'Date': datetime.datetime(2021, 4, 13, 0, 0),
  'Actual': 0.6},
 {'Title': 'Core CPI m/m',
  'Date': datetime.datetime(2021, 4, 13, 0, 0),
  'Actual': 0.3},
 {'Title': 'Core Retail Sales m/m',
  'Date': datetime.datetime(2021, 4, 15, 0, 0),
  'Actual': 8.4},
 {'Title': 'Retail Sales m/m',
  'Date': datetime.datetime(2021, 4, 15, 0, 0),
  'Actual': 9.8},
 {'Title': 'Flash Manufacturing PMI',
  'Date': datetime.datetime(2021, 4, 23, 0, 0),
  'Actual': 60.6}]

In [22]:
import pandas as pd

In [26]:
pd.DataFrame(news_list)

,Title,Date,Actual
0,Final Manufacturing PMI,2021-04-01,59.1
1,ISM Manufacturing PMI,2021-04-01,64.7
2,CPI m/m,2021-04-13,0.6
3,Core CPI m/m,2021-04-13,0.3
4,Core Retail Sales m/m,2021-04-15,8.4
5,Retail Sales m/m,2021-04-15,9.8
6,Flash Manufacturing PMI,2021-04-23,60.6
